<a href="https://colab.research.google.com/github/raphaelrcb/HuggingFace_Workshop/blob/main/Notebooks/HuggingFace_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizagem Com HuggingFace

In [ ]:
!pip install transformers datasets evaluate accelerate

In [ ]:
pip install torch

In [ ]:
pip install tensorflow

# Utilizando a biblioteca Transformers e a função Pipeline

## Análise de Sentimento
Através da Pipeline é possível fazer a Análise de Sentimento de um trecho de texto, dizendo se o texto é positivo ou negativo e a sua confiança. Se não especificado, o classificador carrega um modelo pré-treinado padrão (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english), e seu *tokeneizer*


In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

In [ ]:
classifier("We will be champions tonight")

In [ ]:
classifier("They broke up")

In [ ]:
classifier("They broke up, now I have a chance to go out with her")

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


## Iterando por datasets

A função pipeline() também pode iterar através dos datasets para alguma tarefa necessária. Utilizando reconhecimento de fala automátido por exemplo, é importante carregar o dataset de áudios para que seja possível utilizar no código. Carregaremos o [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14)

In [ ]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

É necessário garantir que a taxa de amostragem do dataset corresponde à taxa de amostragerm do modelo em que foi treinado, no caso: model="facebook/wav2vec2-base-960h"

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

In [ ]:
result = speech_recognizer(dataset[3:7]["audio"])
print([d["text"] for d in result])

## Utilizar outro modelo e _tokenizer_ na pipeline
a pipeline consegue acomodar qualquer modelo do Hub, sendo fácil de adaptar a pipeline para outros casos de uso. Por exemplo, utilizar um modelo que poça usar outra língua.

In [ ]:
model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

Utilize <u>AutoModelForSequenceClassification</u> e <u>AutoTokenizer</u> para carregar um modelo pré-treinado e seu _tokenizer_ associado.



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo.", "Flamengo is world Champion"])

Repetindo com o TensorFlow e outro modelo

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(["Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.", "I hate this", "Yo no creo que ella es tan maligna", "FLAMENGO CAMPEÃO"])

# AutoClass

Uma <u>Autoclass</u> é um atalho que automaticamente busca a arquitetura de um modelo pré-treinado dado seu nome ou caminho. Só é necessário selecionar o Autoclass apropriada para a sua tarefa e classe de pré-processamento associada. Utilizando o exemplo anterior para replicar o resultado da pipeline():


## AutoTokenizer

Um _tokenizer_ é responsável por préprocessar um texto em um _array_ de números como entradas de um modelo. Existem diversas regras que fazem parte de um processo de _tokenização_, incluindo como separar a palavra e até qual nível as palavras devem ser separadas. O mais importante é lembrar que é necessário instanciar o _tokenizer_ com o mesmo nome de modelo para garantir que está sendo usado o mesmo processo de _tokenização_ que o modelo foi pré-treinado.


 https://huggingface.co/docs/transformers/en/tokenizer_summary



In [ ]:
#Carregando um tokenizer com o AutoTokenizer
from transformers import AutoTokenizer

model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
#model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#Passando um texto para o tokenizer
encoding = tokenizer("Flamengo é o maior do mundo.")
print(encoding)

O tokenizer pode retornar um dicionário contendo:


*   <u>input_ids</u>: representações numéricas dos tokens
*   <u>attention_masks</u>: Indica quais tokens devem ser atendidos

O tokenizer também aceita um lista de entradas, preenchendo e truncando o texto para retornart um lote com tamanho uniforme


In [ ]:
#Com PyTorch
pt_batch = tokenizer(
    ["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)
print(pt_batch)

In [ ]:
# Repetindo para o modelo Com TensorFlow

from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tf_batch = tokenizer(
    ["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf",
)
print(tf_batch)


## AutoModel

Transformers entrega uma forma simples e unificada de carregar instâncias pré-treinadas. Isso significa que carregar um <u>AutoModel</u> parece como carregar um <u>AutoTokenizer</u>. A única diferença é selecionar o  <u>AutoModel</u> correto para a tarefa. Para classificação de texto (ou de sequência), deve-se carregar o  <u>AutoModelForSequenceClassification</u>

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
## Para passar o lote pré-processado de entradas diretamente no modelo, deve-se apenas desenpacotar o dicionário adicionando **
# com PyTorch
pt_outputs = pt_model(**pt_batch)
print(pt_outputs)

O modelo tem a saída das ativações finais no atributo `logits`. Aplicando a função softmax, se recebe as probabilidades.:

In [ ]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

In [ ]:
# Repetindo para o TensorFlow

from transformers import TFAutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

## e passando o lote préprocessado direto para o modelo. Você pode passar o tensor como é

tf_outputs = tf_model(tf_batch)
print(tf_outputs)

O modelo tem a saída das ativações finais no atributo `logits`. Aplicando a função softmax, se recebe as probabilidades.:

In [ ]:
import tensorflow as tf

tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions

# Salvar um Modelo

In [ ]:
# PyTorch
## Uma vez que seu modelo está pronto é possível salvá-lo junto de seu tokenizer usando PreTRainedModel.save_pre_trained()

pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

In [ ]:
# E pode carregá-lo novamente com AutoModelForSequenceClassification.from_pretrained()
pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")

In [ ]:
# Para o TensorFlow
## Uma vez que seu modelo está pronto é possível salvá-lo junto de seu tokenizer usando TFPreTRainedModel.save_pre_trained()

tf_save_directory = "./tf_save_pretrained"
tokenizer.save_pretrained(tf_save_directory)
tf_model.save_pretrained(tf_save_directory)

In [ ]:
# E pode carregá-lo novamente com AutoModelForSequenceClassification.from_pretrained()
tf_model = TFAutoModelForSequenceClassification.from_pretrained("./tf_save_pretrained")

Uma característica interessante dos Transformers é a habilidade de salvar e recarregar um modelo tanto como PyTorch ou TensorFlow. o parâmetro `from__pt` e `from_tf` consegue fazer essa conversão


In [ ]:
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)
pt_model = AutoModelForSequenceClassification.from_pretrained(tf_save_directory, from_tf=True)

from transformers import TFAutoModel

tokenizer = AutoTokenizer.from_pretrained(pt_save_directory)
tf_model = TFAutoModelForSequenceClassification.from_pretrained(pt_save_directory, from_pt=True)

# Modelos Customizados

É possível modificar e customizar a classe de configuração de modelos para mudar como ele é construído. A configuração especifíca os atributos do modelo, como o n~umero de camadas ocultas ou _attention heads_. Se começa do zero quando se inicializa um modelo de uma configuração de classe customizada. Os atributos do modelos são inicializados aleatóriamente e deve-se treinar o modelo antes de ser usado para conseguir resultados.

Inicia-se importanto o AutoConfig e então carregando o modelo pré-treinado que se deseja modificar. Dentro de `AutoConfig.from_pretrained()` é possível especificar o atributo que se deseja mudar.

In [ ]:
from transformers import AutoConfig

custom_config = AutoConfig.from_pretrained("distilbert/distilbert-base-uncased", n_heads=12)

In [ ]:
# PyTorch
## Criar um modelo da sua configuração customizada com AutoModel.from_config()

from transformers import AutoModel

custom_model = AutoModel.from_config(custom_config)


In [ ]:
# TensorFlow
## Criar um modelo da sua configuração customizada com TFAutoModel.from_config()

from transformers import TFAutoModel

custom_tf_model = TFAutoModel.from_config(custom_config)


Para mais informações criando um modelo customizado, dê uma olhada em [Criando uma arquitetura customizada](https://huggingface.co/docs/transformers/en/create_a_model).

# Treinando - um loop de treinamento - otimizador no PyTorch

Todos os modelos são padrão `torch.nn.Module` então pode-se utilizá-los em qualquer loop de treinamento. Enquanto é possível escrever o próprio loop de treinamento, os _Transformers_ fornecem uma classe <u>Trainer</u> de treinamento, que contêm o loop de treinamento básico e que adiciona funcionalidades adicionais para catacterísticas como treinamento distribuído, precisão mista e mais.


Dependendo da tarefa, tipicamente se passa os seguintes parâmetros para o <u>Trainer</u>


### 1. Começando com um <u>PreTrainedModel</u> ou um `torch.nn.Module`

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

### 2. <u>TrainingArguments</u> contêm os híper-parâmetros que podem ser alterados, como taxa de aprendizado, tamanho do lote, números de _epochs_ para treinamento. Se não se especificam os valores, os padrões são utilizados.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="path/to/save/folder/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

### 3. Carrega uma classe de pré-processamento como _tokenizer_, _image processor_, _feature extractor_, ou _processor_:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

### 4. Carregar um Dataset.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT

### 5. Criar uma função para _tokenizar_ o dataset e aplique o _dataset_ inteiro com <u>map</u>

In [ ]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])


dataset = dataset.map(tokenize_dataset, batched=True)

### 6. Um <u>DataCollatorWithPadding</u> para criar um lote de exemplos para o seu dataset

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

E junte todas essas classes em um <u>Trainer</u>

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

E quando estiver pronto, chame <u>train()</u> para iniciar o treinamento:

In [ ]:
trainer.train()



---

Para tarefas - como tradução e sumarização - que utilizam modelo sequência a sequência, usa-se no lugar as classes <u>Seq2SeqTrainer</u> <u>Seq2SeqTrainingArguments</u>.

---

pode-se customizar o comportamento do loop de treinamento por <u>_subclassing_</u> os métodos de <u>Trainer</u>. Isso permite que se customize características como função de perda, otimizador e agendador. Isso pode ser visto com mais detalhes na documentação de [Trainer](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/trainer#transformers.Trainer) para quais métodos podem ser <u>subclasses</u>.

# Treinando - com TensorFlow

Todos os modelos são padrão `tf.keras.Model` então podem ser treinados com <u>TensorFLow</u> utilizando a API <u>Keras</u>. _Transformers_ provêm o método <u>prepare_tf_dataset()</u> para carregar seu dataset simplesmente como `tf.data.Dataset` então é possível começar o treinamento imediatamente com o compilador Keras e métodos <u>fit</u>


Dependendo da tarefa, tipicamente se passa os seguintes parâmetros para o <u>Trainer</u>


### 1. Começando com um <u>TFPreTrainedModel</u> ou um `tf.keras.Model`

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

### 2. Carrega uma classe de pré-processamento como _tokenizer_, _image processor_, _feature extractor_, ou _processor_:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

### 3. Criar uma função para _tokenizar_ o dataset

In [ ]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])  # doctest: +SKIP

### 4. Aplique o _tokenizer_ pelo dataset inteiro com <u>map</u> e então passe o dataset e o tokenizer para <u>prepare_tf_dataset()</u>. É possível mudar o tamanho do lote e embaralhar o dataset se preferir.

In [ ]:
dataset = dataset.map(tokenize_dataset)  # doctest: +SKIP
tf_dataset = model.prepare_tf_dataset(
    dataset["train"], batch_size=16, shuffle=True, tokenizer=tokenizer
)  # doctest: +SKIP

### 5. Quando estiver pronto, chame o `compile` e o `fit` para iniciar o treinamento. Nota-se que todos os modelos Transformers têm uma função _task-relevant_ padrão de função de perda, então não é necessário especificar um.

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer='adam')  # No loss argument!
model.fit(tf_dataset)  # doctest: +SKIP